# The Workforce Intelligence Engine: Driving Performance Through Scalable People Analytics  

---

## Introduction  
Organizations are experiencing unprecedented growth in workforce size and geographic spread. While this expansion fuels business opportunities, it also increases the complexity of managing data on people, departments, and costs.  

Without scalable analytics tools, even simple leadership questions about workforce performance or financial planning take days to answer.  

This case study introduces the **Workforce Intelligence Engine** — a framework built with PySpark to unify HR and Finance data into a scalable analytics system.  

---

## Company Background  
**Innovate Solutions Inc.** is a rapidly growing global technology consulting firm. Its workforce has tripled in just two years due to international delivery centers.  

However, this hypergrowth has created challenges:  
- HR and Finance still rely on spreadsheets.  
- Employee and department data is fragmented across systems.  
- Leadership lacks fast, reliable insights into their most valuable asset: people.  

---

## Current Situation  
The absence of a unified analytics engine results in:  

- **Budget Uncertainty**: Finance cannot view real-time salary distribution.  
- **Talent Management Gaps**: HR cannot analyze tenure, hiring patterns, or performance across departments.  
- **Operational Inefficiency**: Answering questions requires manual effort, delaying decision-making.  

---

## Your Role & Objective  
You are a **Senior Data Analyst** on the People Analytics team.  

Your mission is to:  
- Build the foundational **Workforce Intelligence Engine** with PySpark.  
- Ingest HR and Finance data.  
- Transform raw data into actionable intelligence.  

---

## Available Data  

### 1. departments.csv  
- `department_id` — Unique ID for each department.  
- `department_name` — Department name.  
- `location` — City where the department is based.  

### 2. employees.csv  
- `employee_id` — Unique ID for each employee.  
- `employee_name` — Employee full name.  
- `hire_date` — Date the employee was hired.  
- `salary` — Annual salary of the employee.  
- `department_id` — Foreign key linking to departments. 

## 1. Core Imports and Spark Session Initialization  
Before performing any analysis, we need to initialize a **Spark session** and import essential PySpark libraries.  
- `SparkSession` creates and manages the Spark application.  
- `functions` (`F`) provides built-in operations for filtering, aggregation, and transformation.  
- `Window` is used for advanced analytics like ranking and running totals.  

In [5]:
# This section imports necessary libraries and creates a SparkSession, which is the entry point to any Spark functionality.

from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql.window import Window
from pyspark.sql.types import StringType

# pip install pyspark findspark

spark = SparkSession.builder \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .appName("PySparkBeginnerSession") \
    .master("local[*]") \
    .getOrCreate() 

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/29 21:39:01 WARN Utils: Your hostname, Kunals-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.117 instead (on interface en0)
25/09/29 21:39:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/29 21:39:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 2. Read & Load DataFrames  
We will load the datasets provided:  

1. **departments.csv** → Contains details of each department, including name and location.  
2. **employees.csv** → Contains employee-level data, including IDs, names, salaries, hire dates, and department links.  

Both CSV files will be read into **PySpark DataFrames** for analysis.  
We ensure that:  
- Headers are recognized.  
- Data types (integer, string, date) follow the case study specification.  

In [6]:
employees_df = spark.read.csv("employees.csv",header=True)
employees_df.show() 

+-----------+-----------------+----------+------+-------------+
|employee_id|    employee_name| hire_date|salary|department_id|
+-----------+-----------------+----------+------+-------------+
|       1001|     David Harris|2016-09-16| 67812|          105|
|       1002|      Victor Cobb|2021-03-16| 90368|          105|
|       1003|     Joshua Fritz|2019-05-12| 52462|          102|
|       1004|   Shannon Bowers|2020-07-19|109879|          110|
|       1005|     Donna Hunter|2022-12-01| 56019|          110|
|       1006|  Francis Kennedy|2022-04-10|130969|          101|
|       1007| Natasha Anderson|2023-03-07| 78423|          105|
|       1008|   Patricia Scott|2015-08-25|103992|          106|
|       1009|    Wesley Carter|2019-01-07|112309|          103|
|       1010|   Michael Rogers|2022-08-16|132918|          106|
|       1011|    Melanie Heath|2023-02-08| 98144|          102|
|       1012|     Kenneth Hill|2019-03-11|127447|          108|
|       1013|    Lynn Williams|2024-06-1

In [7]:
departments_df = spark.read.csv("departments.csv",header = True)
departments_df.show() 

+-------------+--------------------+-------------------+
|department_id|     department_name|           location|
+-------------+--------------------+-------------------+
|          101|         Media buyer|        Melissastad|
|          102|            Animator|     East Tracyfurt|
|          103|      Hydrogeologist|        Carrchester|
|          104|Fitness centre ma...|        Gilbertfort|
|          105|       Meteorologist|        Brennanbury|
|          106|Research scientis...|       Breannamouth|
|          107|     Hospital doctor|          Yatesport|
|          108| Visual merchandiser|   Port Josephhaven|
|          109|    Technical author|         Davidville|
|          110|Engineer building...|South Catherineberg|
+-------------+--------------------+-------------------+



# --- Part 1: Guided Analytics Session ---  

## Question 1: Find all employees who earn more than $140,000  

In [8]:
employees_df.filter(F.col("salary") > 140000).show() 

+-----------+------------------+----------+------+-------------+
|employee_id|     employee_name| hire_date|salary|department_id|
+-----------+------------------+----------+------+-------------+
|       1017|      Mary Elliott|2025-05-18|141541|          105|
|       1020| Christopher White|2017-07-24|141355|          106|
|       1021|Sophia Nielsen DVM|2017-06-23|141031|          106|
|       1023|     James Jackson|2022-04-20|149130|          109|
|       1036|  Franklin Alvarez|2022-10-14|146824|          104|
|       1039|       Bryan Davis|2017-12-09|146073|          103|
|       1050|      Ethan Martin|2022-08-03|146786|          107|
|       1051|      Joseph Mejia|2024-10-05|149157|          109|
|       1068|    Phillip Miller|2022-04-15|142131|          110|
|       1086|    Carmen Wallace|2019-06-01|142597|          101|
|       1089|     Nathan Montes|2024-12-26|144072|          109|
|       1123|  Christine Howard|2019-02-28|142812|          103|
|       1145|     Shane V

## Question 2: Select only the employee's name, salary, and hire date, and rename `employee_name` to `name`.  

In [6]:
employees_df.select(F.col("employee_name").alias("name"),F.col("salary"),F.col("hire_date")).show() 

+-----------------+------+----------+
|             name|salary| hire_date|
+-----------------+------+----------+
|     David Harris| 67812|2016-09-16|
|      Victor Cobb| 90368|2021-03-16|
|     Joshua Fritz| 52462|2019-05-12|
|   Shannon Bowers|109879|2020-07-19|
|     Donna Hunter| 56019|2022-12-01|
|  Francis Kennedy|130969|2022-04-10|
| Natasha Anderson| 78423|2023-03-07|
|   Patricia Scott|103992|2015-08-25|
|    Wesley Carter|112309|2019-01-07|
|   Michael Rogers|132918|2022-08-16|
|    Melanie Heath| 98144|2023-02-08|
|     Kenneth Hill|127447|2019-03-11|
|    Lynn Williams| 71867|2024-06-15|
| Erin Fitzpatrick|137303|2023-02-17|
|   Matthew Wilson| 53760|2019-07-17|
|  Robert Gonzalez| 57802|2017-05-01|
|     Mary Elliott|141541|2025-05-18|
|      Janet Smith|111431|2018-06-12|
|     Dawn Escobar|131577|2024-04-01|
|Christopher White|141355|2017-07-24|
+-----------------+------+----------+
only showing top 20 rows


## Question 3: Find employees who were hired in the year 2022.  

In [7]:
employees_df.filter(F.year("hire_date") == 2022).show() 

+-----------+--------------------+----------+------+-------------+
|employee_id|       employee_name| hire_date|salary|department_id|
+-----------+--------------------+----------+------+-------------+
|       1005|        Donna Hunter|2022-12-01| 56019|          110|
|       1006|     Francis Kennedy|2022-04-10|130969|          101|
|       1010|      Michael Rogers|2022-08-16|132918|          106|
|       1023|       James Jackson|2022-04-20|149130|          109|
|       1036|    Franklin Alvarez|2022-10-14|146824|          104|
|       1045|        Hannah Smith|2022-01-16|118955|          103|
|       1048|      Nicolas Thomas|2022-09-18|102869|          110|
|       1050|        Ethan Martin|2022-08-03|146786|          107|
|       1053|       Rachel Nelson|2022-09-06|122155|          107|
|       1057|Mrs. Donna Rodriguez|2022-04-25| 78103|          104|
|       1063|          Steven May|2022-11-17|105838|          110|
|       1068|      Phillip Miller|2022-04-15|142131|          

## Question 4: List employees ordered by their hire date, with the most recent hires first.  

In [8]:
employees_df.orderBy(F.col("hire_date").desc()).show(5) 

+-----------+---------------+----------+------+-------------+
|employee_id|  employee_name| hire_date|salary|department_id|
+-----------+---------------+----------+------+-------------+
|       1481|   Brian Reeves|2025-08-21| 84051|          103|
|       1035| Shannon Werner|2025-08-12|115700|          107|
|       1296|   Jason Barron|2025-07-21| 51478|          110|
|       1392|George Peterson|2025-07-05| 88778|          104|
|       1370|    Scott Perez|2025-07-02|106583|          108|
+-----------+---------------+----------+------+-------------+
only showing top 5 rows


## Question 5: How many employees are in each department?  

In [9]:
emp_dept_df = employees_df \
    .join(departments_df,departments_df.department_id == employees_df.department_id,"inner") \
    .drop(departments_df.department_id)
emp_dept_df.show() 

+-----------+-----------------+----------+------+-------------+--------------------+-------------------+
|employee_id|    employee_name| hire_date|salary|department_id|     department_name|           location|
+-----------+-----------------+----------+------+-------------+--------------------+-------------------+
|       1001|     David Harris|2016-09-16| 67812|          105|       Meteorologist|        Brennanbury|
|       1002|      Victor Cobb|2021-03-16| 90368|          105|       Meteorologist|        Brennanbury|
|       1003|     Joshua Fritz|2019-05-12| 52462|          102|            Animator|     East Tracyfurt|
|       1004|   Shannon Bowers|2020-07-19|109879|          110|Engineer building...|South Catherineberg|
|       1005|     Donna Hunter|2022-12-01| 56019|          110|Engineer building...|South Catherineberg|
|       1006|  Francis Kennedy|2022-04-10|130969|          101|         Media buyer|        Melissastad|
|       1007| Natasha Anderson|2023-03-07| 78423|      

In [10]:
emp_dept_df.groupBy("department_name").count().show() 

+--------------------+-----+
|     department_name|count|
+--------------------+-----+
|      Hydrogeologist|   53|
| Visual merchandiser|   44|
|            Animator|   52|
|Engineer building...|   45|
|       Meteorologist|   62|
|Fitness centre ma...|   48|
|    Technical author|   53|
|     Hospital doctor|   49|
|         Media buyer|   47|
|Research scientis...|   47|
+--------------------+-----+



## Question 6: What is the average, maximum, and minimum salary for each department?  

In [11]:
emp_dept_df.groupBy("department_name").agg(F.round(F.avg("salary"),2).alias("average_salary"),F.max("salary").alias("max_salary"),F.min("salary").alias("min_salary"))    .show() 

+--------------------+--------------+----------+----------+
|     department_name|average_salary|max_salary|min_salary|
+--------------------+--------------+----------+----------+
|            Animator|     101103.46|     98144|    100607|
|Engineer building...|      97810.18|     98708|    102070|
|Fitness centre ma...|      99754.75|     98646|    108829|
|     Hospital doctor|     100757.71|     97331|    102086|
|      Hydrogeologist|      94225.15|     93204|    101330|
|         Media buyer|     100449.13|     98932|    102233|
|       Meteorologist|     104025.95|     98338|    100566|
|Research scientis...|     102398.53|     98000|    103992|
|    Technical author|      98327.02|     97958|    103918|
| Visual merchandiser|     105074.14|     99905|    101836|
+--------------------+--------------+----------+----------+



## Question 7: What is the total salary budget for the entire company?  

In [43]:
employees_df.agg(F.format_number(F.sum("salary"),2).alias("total budget")).show() 

+-------------+
| total budget|
+-------------+
|50,196,170.00|
+-------------+



## Question 8: Find all employees who work in a specific city.

In [45]:
emp_dept_df.filter(F.col("location") == "Brennanbury").show()

+-----------+-----------------+----------+------+-------------+---------------+-----------+
|employee_id|    employee_name| hire_date|salary|department_id|department_name|   location|
+-----------+-----------------+----------+------+-------------+---------------+-----------+
|       1001|     David Harris|2016-09-16| 67812|          105|  Meteorologist|Brennanbury|
|       1002|      Victor Cobb|2021-03-16| 90368|          105|  Meteorologist|Brennanbury|
|       1007| Natasha Anderson|2023-03-07| 78423|          105|  Meteorologist|Brennanbury|
|       1017|     Mary Elliott|2025-05-18|141541|          105|  Meteorologist|Brennanbury|
|       1022|    Patrick Baker|2023-08-28| 78604|          105|  Meteorologist|Brennanbury|
|       1032|   Krystal Ramsey|2020-01-12| 91951|          105|  Meteorologist|Brennanbury|
|       1037| Charles Phillips|2024-07-03| 89464|          105|  Meteorologist|Brennanbury|
|       1058|   Michelle David|2018-08-25|136750|          105|  Meteorologist|B

## Question 9: Add a new column to show each employee's salary as a percentage of their department's average salary.  

In [55]:
dept_avg_salary_df = emp_dept_df.groupBy("department_id").agg(F.avg("salary").alias("dept_avg_salary"))

emp_avg_df = emp_dept_df.join(dept_avg_salary_df,"department_id","left")

emp_avg_df = emp_avg_df.withColumn("salary_vs_dept_avg",F.col("salary")/F.col("dept_avg_salary")*100)

emp_avg_df.select("employee_name","department_name","salary","dept_avg_salary","salary_vs_dept_avg").show()

+-----------------+--------------------+------+------------------+------------------+
|    employee_name|     department_name|salary|   dept_avg_salary|salary_vs_dept_avg|
+-----------------+--------------------+------+------------------+------------------+
|     David Harris|       Meteorologist| 67812|104025.95161290323| 65.18757958815797|
|      Victor Cobb|       Meteorologist| 90368|104025.95161290323|  86.8706304521716|
|     Joshua Fritz|            Animator| 52462|101103.46153846153|  51.8894202055016|
|   Shannon Bowers|Engineer building...|109879| 97810.17777777778|112.33902493219293|
|     Donna Hunter|Engineer building...| 56019| 97810.17777777778| 57.27318084143936|
|  Francis Kennedy|         Media buyer|130969|100449.12765957447|130.38341203306257|
| Natasha Anderson|       Meteorologist| 78423|104025.95161290323| 75.38791886453892|
|   Patricia Scott|Research scientis...|103992|102398.53191489361|101.55614348693081|
|    Wesley Carter|      Hydrogeologist|112309| 94225.

## Question 10: Calculate the number of years each employee has been with the company.  

In [58]:
employees_df.withColumn("years_of_service",F.round(F.datediff(F.current_date(),F.col("hire_date"))/365,2)).show() 

+-----------+-----------------+----------+------+-------------+---------------+----------------+
|employee_id|    employee_name| hire_date|salary|department_id|salary_category|years_of_service|
+-----------+-----------------+----------+------+-------------+---------------+----------------+
|       1001|     David Harris|2016-09-16| 67812|          105|         Medium|            9.04|
|       1002|      Victor Cobb|2021-03-16| 90368|          105|         Medium|            4.54|
|       1003|     Joshua Fritz|2019-05-12| 52462|          102|         Medium|            6.38|
|       1004|   Shannon Bowers|2020-07-19|109879|          110|           High|            5.19|
|       1005|     Donna Hunter|2022-12-01| 56019|          110|         Medium|            2.82|
|       1006|  Francis Kennedy|2022-04-10|130969|          101|           High|            3.47|
|       1007| Natasha Anderson|2023-03-07| 78423|          105|         Medium|            2.56|
|       1008|   Patricia Scott

## Question 11: Find the first and last hire date for each department. 

In [66]:
emp_dept_df.groupBy("department_name").agg(F.min("hire_date").alias("first_hire"),F.max("hire_date").alias("last_hire")).show()

+--------------------+----------+----------+
|     department_name|first_hire| last_hire|
+--------------------+----------+----------+
|            Animator|2015-12-06|2025-02-11|
|Engineer building...|2015-11-16|2025-07-21|
|Fitness centre ma...|2015-09-28|2025-07-05|
|     Hospital doctor|2015-09-02|2025-08-12|
|      Hydrogeologist|2015-09-27|2025-08-21|
|         Media buyer|2016-01-13|2025-06-08|
|       Meteorologist|2015-09-01|2025-06-01|
|Research scientis...|2015-08-25|2025-06-12|
|    Technical author|2015-08-30|2024-12-26|
| Visual merchandiser|2015-09-07|2025-07-02|
+--------------------+----------+----------+



## Question 12: Rank employees within each department based on their salary (highest first). 

In [12]:
windowparameter = Window.partitionBy("department_id").orderBy(F.col("salary").desc())

employees_df.withColumn("rank",F.rank().over(windowparameter)).show(10) 

+-----------+-----------------+----------+------+-------------+----+
|employee_id|    employee_name| hire_date|salary|department_id|rank|
+-----------+-----------------+----------+------+-------------+----+
|       1173|Christopher Davis|2021-10-28| 98932|          101|   1|
|       1302| Michael Anderson|2022-08-03| 94001|          101|   2|
|       1352|      Amy Elliott|2024-08-18| 92042|          101|   3|
|       1040|     Steven Jones|2016-08-28| 92015|          101|   4|
|       1349|      Mark Hebert|2022-07-05| 91714|          101|   5|
|       1110|     Marie Taylor|2023-10-27| 88260|          101|   6|
|       1495|       Amy Butler|2025-02-12| 87174|          101|   7|
|       1210|    Daniel Barnes|2018-03-17| 86921|          101|   8|
|       1279|    Jose Robinson|2023-09-06| 85974|          101|   9|
|       1162|      Jared Berry|2022-07-26| 83037|          101|  10|
+-----------+-----------------+----------+------+-------------+----+
only showing top 10 rows


## Question 13: Calculate a running total of salaries within each department, ordered by hire date.  

In [14]:
dept_window = Window.partitionBy("department_id").orderBy("hire_date").rowsBetween(Window.unboundedPreceding, Window.currentRow)

employees_with_running_total = employees_df.withColumn("running_total_salary",F.sum("salary").over(dept_window))

employees_with_running_total.select("employee_id", "employee_name", "department_id", "hire_date", "salary", "running_total_salary").show()

+-----------+---------------+-------------+----------+------+--------------------+
|employee_id|  employee_name|department_id| hire_date|salary|running_total_salary|
+-----------+---------------+-------------+----------+------+--------------------+
|       1305|Ashley Reynolds|          101|2016-01-13|115809|            115809.0|
|       1464|  Kristin Ellis|          101|2016-08-25| 68446|            184255.0|
|       1040|   Steven Jones|          101|2016-08-28| 92015|            276270.0|
|       1163|   Lauren Lloyd|          101|2016-09-23| 80824|            357094.0|
|       1165|  Michael Booth|          101|2017-01-02| 58107|            415201.0|
|       1237|   Barbara Hall|          101|2017-02-05| 50431|            465632.0|
|       1250|   Alison Jones|          101|2017-03-22| 74724|            540356.0|
|       1431|     Mark Lyons|          101|2017-05-21| 55250|            595606.0|
|       1151|  Krystal Nunez|          101|2017-09-11|146549|            742155.0|
|   

## Question 14: Create a new column to categorize employees as 'High', 'Medium', or 'Low' earners based on salary.  

In [10]:
employees_df.withColumn("salary_category",F.when(F.col("salary")>=100000,"High").when(F.col("salary")>=75000,"Medium").otherwise("Low")).show()

+-----------+-----------------+----------+------+-------------+---------------+
|employee_id|    employee_name| hire_date|salary|department_id|salary_category|
+-----------+-----------------+----------+------+-------------+---------------+
|       1001|     David Harris|2016-09-16| 67812|          105|            Low|
|       1002|      Victor Cobb|2021-03-16| 90368|          105|         Medium|
|       1003|     Joshua Fritz|2019-05-12| 52462|          102|            Low|
|       1004|   Shannon Bowers|2020-07-19|109879|          110|           High|
|       1005|     Donna Hunter|2022-12-01| 56019|          110|            Low|
|       1006|  Francis Kennedy|2022-04-10|130969|          101|           High|
|       1007| Natasha Anderson|2023-03-07| 78423|          105|         Medium|
|       1008|   Patricia Scott|2015-08-25|103992|          106|           High|
|       1009|    Wesley Carter|2019-01-07|112309|          103|           High|
|       1010|   Michael Rogers|2022-08-1

## Question 15: Find all employees whose name contains the letter 'a' (case-insensitive).  

In [11]:
employees_df.filter(F.col("employee_name").like("%a%")).show()

+-----------+------------------+----------+------+-------------+
|employee_id|     employee_name| hire_date|salary|department_id|
+-----------+------------------+----------+------+-------------+
|       1001|      David Harris|2016-09-16| 67812|          105|
|       1003|      Joshua Fritz|2019-05-12| 52462|          102|
|       1004|    Shannon Bowers|2020-07-19|109879|          110|
|       1005|      Donna Hunter|2022-12-01| 56019|          110|
|       1006|   Francis Kennedy|2022-04-10|130969|          101|
|       1007|  Natasha Anderson|2023-03-07| 78423|          105|
|       1008|    Patricia Scott|2015-08-25|103992|          106|
|       1009|     Wesley Carter|2019-01-07|112309|          103|
|       1010|    Michael Rogers|2022-08-16|132918|          106|
|       1011|     Melanie Heath|2023-02-08| 98144|          102|
|       1013|     Lynn Williams|2024-06-15| 71867|          102|
|       1014|  Erin Fitzpatrick|2023-02-17|137303|          101|
|       1015|    Matthew 

## Conclusion  
The **Workforce Intelligence Engine** demonstrates how PySpark can be applied to workforce data for fast, accurate, and scalable insights.  

By completing the guided analytics session, you gain the ability to:  
- Answer leadership’s most pressing HR and Finance questions.  
- Build a foundation for advanced analytics like forecasting and predictive modeling.  
- Create a culture of intelligence where workforce data drives business success.  